This document is meant to explain how model V.3 works. In order to test if the implementation works, I generate to create some fake data according to some probabilities that I also made up. The goal is to omit the parameters and to re-learn them from the generated data via MLE and EM.

The whole network is made up of four variables F (fraud/non-fraud), X (input variables), C (hidden variable that should explain X) and ID (the ID of the person that made the transaction)

The network looks like this: F --> X <-- C <-- ID. X is conditioned on F and C and C on id. 
The joint probability equates to: p(ID)p(F)p(ID|C)p(X|F,C)

In [10]:
import numpy as np
import sys
sys.path.append('../')
import EM_v3 as em

In [11]:
#marginal probabilities
p_ids = np.array([0.1, 0.2, 0.3, 0.2, 0.2])
p_f = np.array([.7, .3])

#p(C|ID)
c_given_id = np.array([[0.1, 0.4, 0.4, 0.6, 0.3],
              [0.1, 0.4, 0.3, 0.2, 0.3],
              [0.8, 0.2, 0.3, 0.2, 0.4]])
print "Assert that the conditional probability formulation is correct, by checking that for every id, the probabilities of c sum to 1."
print np.sum(c_given_id, axis=0)

p_c = np.sum(p_ids*c_given_id, axis=1)

p_c_f = p_c[:, np.newaxis]*p_f[:, np.newaxis].T

#p(X|F,ID)
x_given_c_f = np.array([[[ 0.2,  0.2],
               [ 0.2,  0.2],
               [ 0.5,  0.1]],

               [[ 0.15,  0.6],
               [ 0.2,  0.1],
               [ 0.2,  0.2]],

               [[0.1, 0.1],
                [0.2, 0.6],
                [0.2, 0.2]],

               [[0.55, 0.1],
                [0.4, 0.1],
                [0.1, 0.5]]
               ])
print "marginal probability for c: "
print p_c
print "joint probability for c and f: "
print p_c_f
print "Assert that the conditional probability formulation is correct, by checking that for every f, c, the probabilities of x sum to 1."
print np.sum(x_given_c_f, axis=0)

Assert that the conditional probability formulation is correct, by checking that for every id, the probabilities of c sum to 1.
[ 1.  1.  1.  1.  1.]
marginal probability for c: 
[ 0.39  0.28  0.33]
joint probability for c and f: 
[[ 0.273  0.117]
 [ 0.196  0.084]
 [ 0.231  0.099]]
Assert that the conditional probability formulation is correct, by checking that for every f, c, the probabilities of x sum to 1.
[[ 1.  1.]
 [ 1.  1.]
 [ 1.  1.]]


In [12]:
'''
These two methods generate the data with our previously created parameters.
'''
def one_hot(array):
    out = np.zeros((array.shape[0], np.max(array)+1))
    out[np.arange(array.shape[0]), array] = 1
    return out


def simulate(p_ids, p_f, c_given_id, p_c_f, x_given_c_f, n):

    F = np.random.choice(p_f.shape[0], n, p=p_f)

    IDs = np.random.choice(p_ids.shape[0], n, p=p_ids)

    C = np.array([np.random.choice(c_given_id.shape[0], p=c_given_id[:, ID]) for ID in IDs])

    X = np.array((np.array([x_given_c_f[:, c, f] for f, c in zip(F, C)]) > np.random.random((n, x_given_c_f.shape[0]))) * 1.0)

    IDs = one_hot(IDs)
    F = one_hot(F)
    C = one_hot(C)

    return X, F, IDs, C

In [13]:
'''
This is the maximum likelihood estimation function that learns p(X|F)
'''
def mle(X, F):
    p_f = np.mean(F, axis=0)
    x_given_f = np.dot(F.T, X)
    x_given_f /= (np.sum(x_given_f, axis=1)[:, None] )

    return p_f,  x_given_f.T

In [14]:
'''
This part simulates data with made up parameters. 
"C_hidden" is an array with random values that sum to 1 for each data point. 
This is done to "hide" the values of C in order to guess them via EM.
'''
X, F, IDs, C = simulate(p_ids, p_f, c_given_id, p_c_f, x_given_c_f, 10000)

C_hidden = one_hot(np.random.choice(3, 10000, p=(np.arange(3)+7.)/np.sum(np.arange(3)+7.)))

expmax = em.expectation_maximization(0, 0)



p_c_learned, x_given_c_learned, c_given_id_learned = expmax.em_algorithm(C_hidden, X, IDs, 400)

p_f_learned, x_given_f_learned = mle(X, F)

print "assert that x_given_f_learned is true x_given_f"
print "true x_given_f:"
print np.sum(x_given_c_f*p_c_f, axis=1)/p_f
print "learned x_given_f:"
print x_given_f_learned

print "assert that x_given_c_learned is true x_given_c"
print "true x_given_c:"
print np.sum(x_given_c_f*p_c_f, axis=2)/p_c
print "leaned x_given_c:"
print x_given_c_learned

assert that x_given_f_learned is true x_given_f
true x_given_f:
[[ 0.299   0.167 ]
 [ 0.1805  0.328 ]
 [ 0.161   0.273 ]
 [ 0.3595  0.232 ]]
learned x_given_f:
[[ 0.30551953  0.15425356]
 [ 0.18403228  0.3366435 ]
 [ 0.15405678  0.26713009]
 [ 0.35639141  0.24197286]]
assert that x_given_c_learned is true x_given_c
true x_given_c:
[[ 0.2    0.2    0.38 ]
 [ 0.285  0.17   0.2  ]
 [ 0.1    0.32   0.2  ]
 [ 0.415  0.31   0.22 ]]
leaned x_given_c:
[[ 0.31460139  0.33686354  0.17690271]
 [ 0.13120109  0.44534819  0.16258303]
 [ 0.38754802  0.10560747  0.11054835]
 [ 0.1666495   0.1121808   0.54996591]]


The current state of affairs is that the EM algorithm needs to be modified a little to learn X <-- C <-- ID. Currently parameters p(X|C) are a bit off as you can see. The next step which I am working on is to infer p(F|X, ID).

update: 
# E-step

With N data points the dimensions of the input matrix X is (N, D), for C the dimensions are (N, K) and for id the dimensions are (N, (number of id's)).

In the original EM algorithm the E-step was defined as:
$p(C \mid X) = \frac{p(X \mid C) p(C)}{\sum_{k=1}^{K} p(X \mid c_k) p(c_k))} = \frac{p(X \mid C) p(C)}{p(X)}$

where $p(X \mid C) = \prod_{d}^{D} p(x_d \mid C)$

Since we now have a joint probability defined as $p(X \mid C) p(C \mid ID) p(ID)$, we had to modify the E-step such that we find $p(C \mid X, ID)$: 

Since the id is known for each transaction we can multiply $ ID \cdot p(C \mid ID) $, where ID is was the list of id's for each transaction. This results in a (N, K) matrix where for each N we have the associated probability $p(C \mid ID = id)$. I am not entirely sure about this. However, I think that this allows me to replace $p(X \mid C) p(C \mid ID) p(ID)$ with $p(X \mid C) p(C \mid ID =id)$  

The toal E-step update rule is then 
$\frac{p(X \mid C) p(C \mid ID =id)}{\sum_{k=1}^{K} p(X \mid c_k) p(c_k \mid ID =id)}$

# M-step

The M step is very straightforward. 
$p(id)$ is known a priori (just count how many times each id occurs and divide by total amount of transactions).

$p(X|C)$ is calculated by dividing $\frac{\text{# co-occurences of x_d and c_k}}{\text{# occurences of c_k}}$.

$p(C|ID)$ is calculated by dividing $\frac{\text{# co-occurences of c_k and ID=id}}{p(id)}$